In [1]:
import ebest_api as eba
# import auth
import redis
from datetime import datetime
import threading
import logging
from time import gmtime, strftime
import pandas as pd
import time
import requests
import urllib3
from urllib3.exceptions import InsecureRequestWarning
urllib3.disable_warnings(InsecureRequestWarning)


In [2]:
today_date = datetime.now().strftime('%Y-%m-%d')

filename = './신세계_'+ today_date+'.log'

logging.basicConfig(filename=filename,filemode='a+',level=logging.INFO)

ACCESS_TOKEN=eba.make_token()
acc = eba.acc_info(ACCESS_TOKEN)

### 매수(obj_CpTrade_CpTd0311)

In [3]:
balance=acc['CSPAQ12300OutBlock2'].get('MnyOrdAbleAmt')
balance

495748625

In [4]:
def log_it(now_time, info):
    o = str(now_time) +' '+ str(info)
    logging.info(o)

In [5]:
## redis 받는 행동
def get_redis(get_channel_name):

    r = redis.Redis(host='34.64.37.236', port=6379, db=0)

    s = r.pubsub()
    s.subscribe(get_channel_name)

    i = 0
    while True :
        res = s.get_message()
        for message in s.listen():
            data = message.get("data")
            if isinstance(data, bytes):
                i=i+1
                break

        if i==1:
            break

    data = str(data,'utf-8')
    return data

In [6]:
## redis 보내는 행동
def send_redis(send_channel_name, info): # balance_channel으로 할거임

    r = redis.Redis(host='34.64.37.236', port=6379, db=0)

    # print(info)

    # publish
    r.publish(send_channel_name, info)#, perf_rate)
    # r.publish(send_channel_name, perf_rate)

In [7]:
## 찬이 redis 보내는 행동
def chan_send_redis(send_channel_name, info): # balance_channel으로 할거임

    r = redis.Redis(host='34.64.63.106', port=6379, db=0)

    r.publish(send_channel_name, info)

In [8]:
# eba.order(ACCESS_TOKEN,'A004170','buy',1)

In [9]:
# k=eba.trade_order(ACCESS_TOKEN)
# t=k.get('CSPAQ13700OutBlock3')
# all=[]
# for i,li in enumerate(t):
#     if li.get('IsuNm')=='신세계':
#         all.append(li)
        
# real_close=str(int(float(all[-1].get('ExecPrc'))))*int(all[-1].get('ExecQty'))
# print('수정된 real_close:'+real_close)

In [10]:
def autobuy(stock):
    ACCESS_TOKEN=eba.make_token()
    all=[]
    now_time5 = datetime.now().strftime('%H%M')
    while int(now_time5) <=1520:
        now_time5 = datetime.now().strftime('%H%M')

        print("=======================================")
        print(stock)
        now_min1 = datetime.now().strftime('%M')
        now_time = datetime.now().strftime('%H:%M:%S')    
        acc = eba.acc_info(ACCESS_TOKEN)
        
        if 'CSPAQ12300OutBlock2' in acc.keys():
            print("[API] 잔고를 가져옵니다")
            balance=acc['CSPAQ12300OutBlock2'].get('MnyOrdAbleAmt')
        
        print("현재시각은 {}입니다".format(now_time))
        print('남은 잔고는 {}입니다'.format(balance))
        send_redis(stock+'balance_channel', balance)
        log_it(now_time, balance)
        print("[Redis] action을 기다리는 중입니다")
        action = get_redis(stock+'action_channel')
        log_it(now_time, action)
        
        print("[API] 체결단가를 가져오는중..")
        while True:
            ACCESS_TOKEN=eba.make_token()
            trade = eba.trade_order(ACCESS_TOKEN)
        # 'CSPAQ13700OutBlock3' not in trade.keys():

            if 'CSPAQ13700OutBlock3' in trade.keys():  
                t=trade.get('CSPAQ13700OutBlock3')
                for i,li in enumerate(t):
                    if li.get('IsuNm')=='신세계':
                        all.append(li)

                print("[API] 체결 단가를 가져옵니다")
                real_close1=str(int(float(all[-1].get('ExecPrc'))))*int(all[-1].get('ExecQty'))
                print('수정된1 real_close:'+real_close1)
                break



            # if 'CSPAQ13700OutBlock3' in trade.keys(): 
            #     print("[API] 체결단가를 가져옵니다")
            #     trade = trade['CSPAQ13700OutBlock3']
            #     real_close1 = str(int(float(trade[0]['ExecPrc'])))# * int(trade[0]['ExecQty']))
            #     print('수정된1 real_close:'+real_close1)
            #     break


        # k=eba.trade_order(ACCESS_TOKEN)
        # t=k.get('CSPAQ13700OutBlock3')
        # if 'CSPAQ13700OutBlock3' in acc.keys():
        #     for i,li in enumerate(t):
        #         # if li.get('IsuNm')=='신세계':
        #             all.append(li)
        #     real_close=str(int(float(all[0].get('ExecPrc'))))*int(all[0].get('ExecQty'))
        #     print('수정된 real_close:'+real_close)



        while action=='0':
            real_close = real_close1
            now_time = datetime.now().strftime('%H:%M:%S')
            print('현재시각 {} 홀드합니다'.format(now_time))
            # print('남은 잔고는 {}입니다'.format(balance))
            time.sleep(20)
            now_time = datetime.now().strftime('%H:%M:%S')
            print('현재시각 {} 홀드합니다'.format(now_time))
            send_redis(stock+'real_close', real_close)
            print('[Redis] 보내는 종가는 {}입니다'.format(real_close))
            time.sleep(1)
            send_redis(stock+'balance_channel', balance)
            print('[Redis] 보내는 잔고는 {}입니다'.format(balance))
            log_it(now_time, balance)
            # time.sleep(5)
            log_it(now_time, real_close)
            # time.sleep(10)
            print("[Redis] action을 기다리는 중입니다")
            action = get_redis(stock+'action_channel')
            log_it(now_time, '홀드')

            if action!='0':
                break

##################### 매매하는 부분
        if action =='2':
            ACCESS_TOKEN=eba.make_token()
            now_time = datetime.now().strftime('%H:%M:%S')
            print('현재시각 {} 매수합니다'.format(now_time))
            log_it(now_time, '매수')
            eba.order(ACCESS_TOKEN,'A'+stock,'buy',1)

        if action =='1':
            ACCESS_TOKEN=eba.make_token()
            now_time = datetime.now().strftime('%H:%M:%S')
            print('현재시각 {} 매도합니다'.format(now_time))
            log_it(now_time, '매도')
            eba.order(ACCESS_TOKEN,'A'+stock,'sell',1)
####################     


        if 'CSPAQ12300OutBlock2' in acc.keys():
            print("[API] 잔고를 가져옵니다")
            balance=acc['CSPAQ12300OutBlock2'].get('MnyOrdAbleAmt')


        # k=eba.trade_order(ACCESS_TOKEN)
        # t=k.get('CSPAQ13700OutBlock3')

        # t=trade.get('CSPAQ13700OutBlock3')

        # if 'CSPAQ13700OutBlock3' in acc.keys():
        #     for i,li in enumerate(t):
        #         if li.get('IsuNm')=='신세계':
        #             all.append(li)


        print("[API] 체결단가를 가져오는중..")
        while True:
            ACCESS_TOKEN=eba.make_token()
            trade = eba.trade_order(ACCESS_TOKEN)
        # 'CSPAQ13700OutBlock3' not in trade.keys():
           


            if 'CSPAQ13700OutBlock3' in trade.keys():  
                t=trade.get('CSPAQ13700OutBlock3')
                for i,li in enumerate(t):
                    if li.get('IsuNm')=='신세계':
                        all.append(li)

                print("[API] 체결 단가를 가져옵니다")
                real_close2=str(int(float(all[-1].get('ExecPrc'))))*int(all[-1].get('ExecQty'))


                # trade = trade['CSPAQ13700OutBlock3']
                # real_close2 = str(int(float(trade[0]['ExecPrc'])))# * int(trade[0]['ExecQty']))
                print('수정된2 real_close:'+real_close2)
                break
       
       
        if real_close2=='0':
            real_close = real_close1
        else:
            real_close = real_close2


        # while True:
        #     if action =='2':
        #         now_time = datetime.now().strftime('%H:%M:%S')
        #         print('현재시각 {} 매수합니다'.format(now_time))
        #         log_it(now_time, '매수')
        #         eba.order(ACCESS_TOKEN,'A'+stock,'buy',1)

        #     if action =='1':
        #         now_time = datetime.now().strftime('%H:%M:%S')
        #         print('현재시각 {} 매도합니다'.format(now_time))
        #         log_it(now_time, '매도')
        #         eba.order(ACCESS_TOKEN,'A'+stock,'sell',1)


        #     if real_close != '0'
        #         break


        # k=eba.trade_order(ACCESS_TOKEN)
        # t=k.get('CSPAQ13700OutBlock3')

        # # if 'CSPAQ13700OutBlock3' in acc.keys():
        # for i,li in enumerate(t):
        #     if li.get('IsuNm')=='신세계':
        #         all.append(li)
        # real_close=str(int(float(all[0].get('ExecPrc'))))*int(all[0].get('ExecQty'))
        # print('수정된 real_close:'+real_close)


        print('남은 잔고는 {}입니다 보냅니다'.format(balance))
        time.sleep(0.1)
        send_redis(stock+'balance_channel', balance)
        now_time = datetime.now().strftime('%H:%M:%S')
        log_it(now_time, balance)
        print('체결단가는 {}입니다 학습하게 보냅니다'.format(real_close))
        time.sleep(0.1)
        send_redis(stock+'real_close', real_close)
        now_time = datetime.now().strftime('%H:%M:%S')
        log_it(now_time, real_close)

        # now_time = datetime.now().strftime('%H%M')
        # if int(now_time) >1520: #15시20분이면 종료
        #     break



In [11]:
autobuy('004170')


004170
[API] 잔고를 가져옵니다
현재시각은 12:42:08입니다
남은 잔고는 495748625입니다
[Redis] action을 기다리는 중입니다
[API] 체결단가를 가져오는중..
[API] 체결 단가를 가져옵니다
수정된1 real_close:191000
현재시각 12:42:26 매수합니다
[API] 잔고를 가져옵니다
[API] 체결단가를 가져오는중..
[API] 체결 단가를 가져옵니다
수정된2 real_close:191000
남은 잔고는 495748625입니다 보냅니다
체결단가는 191000입니다 학습하게 보냅니다
004170
[API] 잔고를 가져옵니다
현재시각은 12:42:27입니다
남은 잔고는 495690725입니다
[Redis] action을 기다리는 중입니다
[API] 체결단가를 가져오는중..
[API] 체결 단가를 가져옵니다
수정된1 real_close:191000
현재시각 12:42:32 매수합니다
[API] 잔고를 가져옵니다
[API] 체결단가를 가져오는중..
[API] 체결 단가를 가져옵니다
수정된2 real_close:191100
남은 잔고는 495690725입니다 보냅니다
체결단가는 191100입니다 학습하게 보냅니다
004170
[API] 잔고를 가져옵니다
현재시각은 12:42:33입니다
남은 잔고는 495652445입니다
[Redis] action을 기다리는 중입니다
[API] 체결단가를 가져오는중..
[API] 체결 단가를 가져옵니다
수정된1 real_close:191100
현재시각 12:43:06 매수합니다
[API] 잔고를 가져옵니다
[API] 체결단가를 가져오는중..
[API] 체결 단가를 가져옵니다
수정된2 real_close:191000
남은 잔고는 495652445입니다 보냅니다
체결단가는 191000입니다 학습하게 보냅니다
004170
[API] 잔고를 가져옵니다
현재시각은 12:43:07입니다
남은 잔고는 495588165입니다
[Redis] action을 기다리는 중입니다
[API] 체결단가를 가져오는중..

In [65]:
k=eba.trade_order(ACCESS_TOKEN)
t=k.get('CSPAQ13700OutBlock3')

In [66]:
t

[{'OrdDt': '20230731',
  'MgmtBrnNo': '',
  'OrdMktCode': '',
  'OrdNo': 10392,
  'OrgOrdNo': 0,
  'IsuNo': '',
  'IsuNm': '이마트',
  'BnsTpCode': '2',
  'BnsTpNm': '0',
  'OrdPtnCode': '',
  'OrdPtnNm': '(KSE)현금매수',
  'OrdTrxPtnCode': 0,
  'OrdTrxPtnNm': '정상',
  'MrcTpCode': '0',
  'MrcTpNm': '정상',
  'MrcQty': 0,
  'MrcAbleQty': 0,
  'OrdQty': 1,
  'OrdPrc': '0.00',
  'ExecQty': 1,
  'ExecPrc': '75400.00',
  'ExecTrxTime': '10:56:23',
  'LastExecTime': '10:56:23',
  'OrdprcPtnCode': '03',
  'OrdprcPtnNm': '시장가',
  'OrdCndiTpCode': '0',
  'AllExecQty': 1,
  'RegCommdaCode': '40',
  'CommdaNm': '',
  'MbrNo': '',
  'RsvOrdYn': '0',
  'LoanDt': '',
  'OrdTime': '10:56:20',
  'OpDrtnNo': '',
  'OdrrId': 'rnddl47'},
 {'OrdDt': '20230731',
  'MgmtBrnNo': '',
  'OrdMktCode': '',
  'OrdNo': 10358,
  'OrgOrdNo': 0,
  'IsuNo': '',
  'IsuNm': '이마트',
  'BnsTpCode': '2',
  'BnsTpNm': '0',
  'OrdPtnCode': '',
  'OrdPtnNm': '(KSE)현금매수',
  'OrdTrxPtnCode': 0,
  'OrdTrxPtnNm': '정상',
  'MrcTpCode': '0',


In [ ]:
# # def aaa():
# dicttt = c.get_trade_history()
# diclist = dicttt.get('data')
# # diclist

In [ ]:

# li = []
# for data in diclist:
#     # print(data)
#     if data.get('종목이름') == '신세계':
#         li.append(data)
    

    
# print(li)
# real_close = li[len(li)-1].get('체결단가')*li[len(li)-1].get('체결수량')
# real_close = str(real_close)
# print("수정한 real_close {}".format(real_close))


[{'상품관리구분코드': '10', '주문번호': 1845, '원주문번호': 0, '종목코드': 'A004170', '종목이름': '신세계', '주문내용': '정규장현금매도', '주문호가구분코드내용': '시장가', '주문수량': 1, '주문단가': 0, '총체결수량': 1, '체결수량': 1, '체결단가': 191800, '확인수량': 0, '정정취소구분내용 ': '정상주문', '거부사유내용': '', '채권매수일': '', '거래세과세구분내용': '과 세', '현금신용대용구분내용': '현금', '주문입력매체코드내용': 'CYBOS 5', '정정취소가능수량': 0, '매매구분': '매도'}, {'상품관리구분코드': '10', '주문번호': 2025, '원주문번호': 0, '종목코드': 'A004170', '종목이름': '신세계', '주문내용': '정규장현금매도', '주문호가구분코드내용': '시장가', '주문수량': 1, '주문단가': 0, '총체결수량': 1, '체결수량': 1, '체결단가': 191700, '확인수량': 0, '정정취소구분내용 ': '정상주문', '거부사유내용': '', '채권매수일': '', '거래세과세구분내용': '과 세', '현금신용대용구분내용': '현금', '주문입력매체코드내용': 'CYBOS 5', '정정취소가능수량': 0, '매매구분': '매도'}, {'상품관리구분코드': '10', '주문번호': 2190, '원주문번호': 0, '종목코드': 'A004170', '종목이름': '신세계', '주문내용': '정규장현금매도', '주문호가구분코드내용': '시장가', '주문수량': 1, '주문단가': 0, '총체결수량': 1, '체결수량': 1, '체결단가': 190900, '확인수량': 0, '정정취소구분내용 ': '정상주문', '거부사유내용': '', '채권매수일': '', '거래세과세구분내용': '과 세', '현금신용대용구분내용': '현금', '주문입력매체코드내용': 'CYBOS 5', '정정취소가능수량': 0, '매매구분': '매도'

In [ ]:
# aaa()

In [ ]:
# c.order(action=1,code='004170',amount=1)

----